##### NULLIF()
- The **NULLIF()** function returns **NULL** if **two expressions are equal**, otherwise it returns the **first expression**.

##### Syntax

     NULLIF(expr1, expr2)

     NULL if expr1 = expr2
     expr1 if expr1 ≠ expr2

##### 1) Basic Comparison

In [0]:
%sql
-- Compare two expressions
SELECT NULLIF(10, 10) AS result1,                          -- returns NULL  (since 10 = 10)
       NULLIF(10, 20) AS result2,                          -- returns 10    (since 10 ≠ 20)
       NULLIF('Hello', 'Hello') AS result3,                -- returns NULL  (since Hello = Hello)
       NULLIF('Hello', 'world') AS result4,                -- returns Hello (since Hello ≠ world)
       NULLIF('2017-08-25', '2017-08-25') AS result5;      -- returns NULL  (since 2017-08-25 = 2017-08-25)

result1,result2,result3,result4,result5
null,10,null,Hello,null


##### 2) Avoiding Divide by Zero

In [0]:
%sql
DROP TABLE IF EXISTS tblEmployeesNullIF;
CREATE TABLE tblEmployeesNullIF (
    EmpID INT PRIMARY KEY,
    Name VARCHAR(50),
    Salary INT,
    YearsWorked INT,
    department VARCHAR(10),
    bonus INT
);

INSERT INTO tblEmployeesNullIF (EmpID, Name, Salary, YearsWorked, department, bonus)
VALUES
(1, 'Albert', 50000, 5, 'HR', 5),     -- Normal case
(2, 'Bobby', 60000, 0, 'ADMIN', 0),       -- Divide by zero avoided
(3, 'Charan', 45000, 9, 'FINACE', 6),   -- Normal case
(4, 'David', 30000, NULL, 'HR', 9),  -- NULL YearsWorked
(5, 'Eshwar', 75000, 15, 'ACCOUNTS', 4);      -- Normal case

SELECT * FROM tblEmployeesNullIF;

EmpID,Name,Salary,YearsWorked,department,bonus
1,Albert,50000,5,HR,5
2,Bobby,60000,0,ADMIN,0
3,Charan,45000,9,FINACE,6
4,David,30000,null,HR,9
5,Eshwar,75000,15,ACCOUNTS,4


In [0]:
%sql
SELECT Salary / YearsWorked AS AvgPerYear
FROM tblEmployeesNullIF;

-- [DIVIDE_BY_ZERO] Division by zero. Use `try_divide` to tolerate divisor being 0 and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22012

- If **YearsWorked = 0**, this causes an **error**.

In [0]:
%sql
SELECT Salary, YearsWorked,
       Salary / NULLIF(YearsWorked, 0) AS AvgPerYear
FROM tblEmployeesNullIF;

Salary,YearsWorked,AvgPerYear
50000,5,10000.0
60000,0,null
45000,9,5000.0
30000,null,null
75000,15,5000.0


- If **YearsWorked = 0**, **NULLIF(YearsWorked, 0) = NULL**, so the **result = NULL instead of error**.

##### 3) Removing Duplicate Values

In [0]:
%sql
SELECT EmpID, Name, Department,
       NULLIF(Department, 'HR') AS DeptCheck
FROM tblEmployeesNullIF;

EmpID,Name,Department,DeptCheck
1,Albert,HR,null
2,Bobby,ADMIN,ADMIN
3,Charan,FINACE,FINACE
4,David,HR,null
5,Eshwar,ACCOUNTS,ACCOUNTS


- If **Department = 'HR'**, returns **NULL**.
- Otherwise, returns **actual Department**.

##### 4) Using in CASE Expressions

In [0]:
%sql
SELECT EmpID, Salary, bonus,
       CASE 
            WHEN NULLIF(bonus, 0) IS NULL THEN 'No Bonus'
            ELSE 'Has Bonus'
       END AS BonusStatus
FROM tblEmployeesNullIF;

EmpID,Salary,bonus,BonusStatus
1,50000,5,Has Bonus
2,60000,0,No Bonus
3,45000,6,Has Bonus
4,30000,9,Has Bonus
5,75000,4,Has Bonus


- If Bonus = 0, **NULLIF(Bonus,0) = NULL** → "No Bonus".
- Otherwise, **"Has Bonus"**.

##### 5) Comparing Two Columns

In [0]:
%sql
SELECT Salary, bonus,
       NULLIF(Salary, bonus) AS SalaryCheck
FROM tblEmployeesNullIF;

Salary,bonus,SalaryCheck
50000,5,50000
60000,0,60000
45000,6,45000
30000,9,30000
75000,4,75000


- If **Salary = Bonus** → returns **NULL**.
- Else → returns **Salary**.

##### 6) Filtering Rows

In [0]:
%sql
SELECT Name, Department
FROM tblEmployeesNullIF
WHERE NULLIF(Name, Department) IS NULL;

Name,Department


- This finds rows where **Name = Department**.

##### 7) NULLIF() with COALESCE & AVG

In [0]:
%sql
DROP TABLE IF EXISTS tblBudgetsNullIf;
CREATE TABLE tblBudgetsNullIf (  
   dept           TINYINT,  
   current_year   DECIMAL(10,2),  
   previous_year  DECIMAL(10,2)  
);  
  
INSERT INTO tblBudgetsNullIf VALUES(1, 100000, 150000);  
INSERT INTO tblBudgetsNullIf VALUES(2, NULL, 300000);  
INSERT INTO tblBudgetsNullIf VALUES(3, 0, 100000);  
INSERT INTO tblBudgetsNullIf VALUES(4, NULL, 150000);  
INSERT INTO tblBudgetsNullIf VALUES(5, 300000, 300000);

SELECT * FROM tblBudgetsNullIf;

dept,current_year,previous_year
1,100000.00,150000.00
5,300000.00,300000.00
3,0.00,100000.00
2,null,300000.00
4,null,150000.00


In [0]:
%sql
SELECT current_year, previous_year,
       NULLIF(current_year, previous_year) AS LastBudget  
FROM tblBudgetsNullIf;

current_year,previous_year,LastBudget
100000.00,150000.00,100000.00
300000.00,300000.00,null
0.00,100000.00,0.00
null,300000.00,null
null,150000.00,null


In [0]:
%sql
SELECT current_year, previous_year,
       COALESCE(current_year, previous_year) AS Colsc_TotalBudget,
       NULLIF(COALESCE(current_year, previous_year), 0.00) AS NullIF_LastBudget  
FROM tblBudgetsNullIf;

current_year,previous_year,Colsc_TotalBudget,NullIF_LastBudget
100000.00,150000.00,100000.00,100000.00
300000.00,300000.00,300000.00,300000.00
0.00,100000.00,0.00,null
null,300000.00,300000.00,300000.00
null,150000.00,150000.00,150000.00


In [0]:
%sql
SELECT AVG(NULLIF(COALESCE(current_year, previous_year), 0.00)) AS Average_Budget
FROM tblBudgetsNullIf;

Average_Budget
212500.000000
